<center><h1>CSCI 3675</h1></center>
<center><h1>Principles of Programming Languages</h1></center>

<center><h6>East Carolina University</h6></center>
<center><h6>Nic Herndon, PhD</h6></center>
<br/>
<center><h6>Haskell - Part 3</h6></center>

<p style="font-size:14px">Material adapted from Erik Meijer’s Functional Programming</p>

## Lesson plan

* Declaring types and classes
* ICEs
* Lazy evaluation
* ICEs
* Summary

# Declaring types and classes
## Type declarations

In Haskell, a new name for an existing type can be defined using a **type declaration**.

```
type String = [Char]
```

String is a synonym for the type `[Char]`.

In [ ]:
type String = [Char]

Type declarations can be used to make other types easier to read. For example, given

```
type Pos = (Int,Int)
```

we can define:

```
origin    :: Pos
origin     = (0,0)

left      :: Pos → Pos
left (x,y) = (x-1,y)
```

In [ ]:
type Pos = (Int,Int)

origin    :: Pos
origin     = (0,0)

left      :: Pos -> Pos
left (x,y) = (x-1,y)

Like function definitions, type declarations can also have **parameters**. For example, given

```
type Pair a = (a,a)
```

we can define:

```
mult      :: Pair Int → Int
mult (m,n) = m*n

copy      :: a → Pair a
copy x     = (x,x)
```

In [ ]:
type Pair a = (a,a)

mult      :: Pair Int -> Int
mult (m,n) = m*n

copy      :: a -> Pair a
copy x     = (x,x)

Type declarations can be nested:

```
-- OK
type Pos   = (Int,Int)

type Trans = Pos → Pos
```

However, they cannot be recursive:
```
-- Not OK
type Tree = (Int,[Tree])
```

In [ ]:
-- OK
type Pos   = (Int,Int)

type Trans = Pos -> Pos

In [ ]:
-- Not OK
type Tree = (Int,[Tree])

## Data declarations

A completely new type can be defined by specifying its values using a **data declaration**.

```
data Bool = False | True
```

`Bool` is a new type, with two new values `False` and `True`.

In [ ]:
data Bool = False | True

Note:
- The two values `False` and `True` are called the **constructors** for the type `Bool`.
- Type and constructor names must begin with an upper-case letter.
- Data declarations are similar to context free grammars. The former specifies the values of a type, the latter the sentences of a language.

Values of new types can be used in the same ways as those of built in types. For example, given

```
data Answer = Yes | No | Unknown
```

we can define:

```
answers     :: [Answer]
answers      = [Yes,No,Unknown]

flip        :: Answer → Answer
flip Yes     = No
flip No      = Yes
flip Unknown = Unknown
```

In [ ]:
data Answer = Yes | No | Unknown

answers     :: [Answer]
answers      = [Yes,No,Unknown]

flip        :: Answer -> Answer
flip Yes     = No
flip No      = Yes
flip Unknown = Unknown

The constructors in a data declaration can also have parameters. For example, given

```
data Shape = Circle Float
           | Rect Float Float
```

we can define:

```
square         :: Float → Shape
square n        = Rect n n

area           :: Shape → Float
area (Circle r) = pi * r^2
area (Rect x y) = x * y
```

In [ ]:
data Shape = Circle Float
           | Rect Float Float

square         :: Float -> Shape
square n        = Rect n n

area           :: Shape -> Float
area (Circle r) = pi * r^2
area (Rect x y) = x * y

Note:

- Shape has values of the form `Circle r` where `r` is a float, and `Rect x y` where `x` and `y` are floats.
- `Circle` and `Rect` can be viewed as **functions** that construct values of type `Shape`:

```
Circle :: Float → Shape

Rect   :: Float → Float → Shape
```

Not surprisingly, data declarations themselves can also have parameters. For example, given

```
data Maybe a = Nothing | Just a
```

we can define:

```
safediv    :: Int → Int → Maybe Int
safediv _ 0 = Nothing
safediv m n = Just (m `div` n)

safehead   :: [a] → Maybe a
safehead [] = Nothing
safehead xs = Just (head xs)
```

In [ ]:
data Maybe a = Nothing | Just a

safediv    :: Int -> Int -> Maybe Int
safediv _ 0 = Nothing
safediv m n = Just (m `div` n)

safehead   :: [a] -> Maybe a
safehead [] = Nothing
safehead xs = Just (head xs)

## Recursive types

In Haskell, new types can be declared in terms of themselves. That is, types can be **recursive**.

```
data Nat = Zero | Succ Nat
```

`Nat` is a new type, with constructors `Zero :: Nat` and `Succ :: Nat → Nat`.

In [ ]:
data Nat = Zero | Succ Nat

Note:
- A value of type `Nat` is either `Zero`, or of the form `Succ n` where `n :: Nat`. That is, `Nat` contains the following infinite sequence of values:

```
Zero
Succ Zero
Succ (Succ Zero)
•
•
•
```

- We can think of values of type `Nat` as **natural numbers**, where `Zero` represents `0`, and `Succ` represents the successor function `1+`.

- For example, the value

```
Succ (Succ (Succ Zero))
```

represents the natural number `1 + (1 + (1 + 0)) = 3`.

Using recursion, it is easy to define functions that convert between values of type `Nat` and `Int`:

```
nat2int         :: Nat → Int
nat2int Zero     = 0
nat2int (Succ n) = 1 + nat2int n

int2nat  :: Int → Nat
int2nat 0 = Zero
int2nat n = Succ (int2nat (n-1))
```

In [ ]:
nat2int         :: Nat -> Int
nat2int Zero     = 0
nat2int (Succ n) = 1 + nat2int n

int2nat  :: Int -> Nat
int2nat 0 = Zero
int2nat n = Succ (int2nat (n-1))

Two naturals can be added by converting them to integers, adding, and then converting back:

```
add    :: Nat → Nat → Nat
add m n = int2nat (nat2int m + nat2int n)
```

However, using recursion the function `add` can be defined without the need for conversions:

```
add Zero     n = n
add (Succ m) n = Succ (add m n)
```

In [ ]:
add    :: Nat -> Nat -> Nat
add m n = int2nat (nat2int m + nat2int n)

add Zero     n = n
add (Succ m) n = Succ (add m n)

For example:

```
add (Succ (Succ Zero)) (Succ Zero)
= Succ (add (Succ Zero) (Succ Zero))
= Succ (Succ (add Zero (Succ Zero))
= Succ (Succ (Succ Zero))
```

Note: 
- The recursive definition for `add` corresponds to the laws `0+n = n` and `(1+m)+n = 1+(m+n)`.

# Lazy evaluation
## Introduction

Up to now, we have not looked in detail at how Haskell expressions are **evaluated**.

In fact they are evaluated using a simple technique that, among other things:

1.  Avoids doing **unnecessary evaluation**

2.  Allows programs to be **more modular**

3.  Allows us to program with **infinite lists**

The evaluation technique is called **lazy evaluation**, and Haskell is a **lazy functional language**.

## Evaluating expressions

Basically, expressions are evaluated or **reduced** by successively **applying definitions** until no further simplification is possible.

For example, given the definition: `square n = n * n`

The expression **`square(3 + 4)`** can be evaluated using the following sequence of reductions:

## Evaluating expressions

```
square n = n * n

square (3 + 4)
= square 7
= 7 * 7
= 49
```

However, this is not the only possible reduction sequence.

**Another** is the following:

```
square (3 + 4)
= (3 + 4) * (3 + 4)
= 7 * (3 + 4)
= 7 * 7
= 49
```

Now we have applied **square** before doing the addition, but the final result is the same.

## FACT

In Haskell, two **different** (but terminating) ways of evaluating **the same** expression will always give **the same** final result.

## Reduction strategies

At each stage during evaluation of an expression, there may be **many** possible subexpressions that can be reduced by applying a definition.

There are two common strategies for deciding which **redex** (**red**ucible sub**ex**pression) to choose:

1. Innermost reduction: An innermost redex is always reduced
2. Outermost reduction: An outermost redex is always reduced

How do the two strategies compare … ?

## Termination

Given the definition: `loop = tail loop`

Let’s evaluate the expression `fst (1, loop)` using these two reduction strategies.

## Innermost reduction

```
fst (1, loop)
= fst (1, tail loop)
= fst (1, tail (tail loop))
= ...
```
This strategy does **not terminate**.

## Outermost reduction

```
fst (1, loop)
= 1
```

This strategy gives a result in **one step**.

**Facts**

- Outermost reduction may give a result when innermost reduction **fails to terminate**.
- For a given expression if there exists **any** reduction sequence that terminates, then outermost reduction **also** terminates, with the **same result**.

## Number of reductions

Innermost
```
square (3 + 4)
= square 7
= 7 * 7
= 49
```
                                 
Outermost
```
square (3 + 4)
= (3 + 4) * (3 + 4)
= 7 * (3 + 4)
= 7 * 7
= 49
```

The outermost version is **inefficient**: the subexpression `3 + 4` is duplicated when **square** is reduced, and so must be reduced twice.

**Fact** Outermost reduction may require **more** steps than innermost reduction.

The problem can be solved by using **pointers** to indicate **sharing** of expressions during evaluation:

<center><img src="images/pointers.png" width="500" height="600" /></center>

This gives a new reduction strategy:

**Lazy evaluation** = Outermost reduction + sharing

**Facts**

- **Never** requires more steps than innermost reduction
- **Haskell** uses lazy evaluation

## Inﬁnite lists

In addition to the termination advantages, using lazy evaluation allows us to program with **infinite lists** of values!

Consider the recursive definition:

```
ones :: [Int]
ones = 1 : ones
```

Unfolding the recursion a few times gives:

```
ones = 1 : ones
     = 1 : 1 : ones
     = 1 : 1 : 1 : ones
```

That is, **ones** is the **infinite list** of 1’s.

Now consider evaluating the expression

```
head ones
```

using **innermost reduction** and **lazy evaluation**.

## Innermost reduction

```
head ones = head (1 : ones)
          = head (1 : 1 : ones) 
          = head (1 : 1 : 1 : ones) 
          = ...
```

In this case, evaluation **does not terminate**.

## Lazy evaluation

```
head ones = head (1 : ones) 
          = 1
```

In this case, evaluation gives the result 1.

That is, using lazy evaluation only the **first** value in the infinite **list** ones is actually produced, since this is all that is required to evaluate the expression **head ones** as a whole.

## Lazy evaluation

In general we have the slogan:

Using lazy evaluation, expressions are only evaluated **as much as required** to produce the final result.

We see now that

```
ones = 1 : ones
```

really only defines a **potentially infinite** list that is only evaluated as much as required by the context in which it is used.

## Modular programming

We can generate **finite** lists by taking elements from infinite lists.

```
> take 5 ones
[1,1,1,1,1]
> take 5 [1..]
[1,2,3,4,5]
```

Lazy evaluation allows us to make programs **more modular**, by separating control from data:

<center><img src="images/modular.png" width="500" height="600" /></center>

Using lazy evaluation, the data is only evaluated as much as required by the control part.

## Example: generating primes

A simple procedure for generating the **infinite list** of **prime numbers** is as follows:

1. Write down the list `2, 3, 4, …` ;
2. Mark the first value `p` in the list as prime;
3. Delete all multiples of `p` from the list;
4. Return to step 2.

## Example: generating primes

The first few steps can be pictured by:

<center><img src="images/primes.png" width="800" height="1200" /></center>

This procedure is known as the “seive of Eratosthenes”, after the Greek mathematician who first described it.

It can be translated **directly** into Haskell:

```
primes :: [Int]
primes = seive [2..]

seive :: [Int] -> [Int]
seive (p:xs) = p : seive [x | x <- xs, x `mod` p /= 0]
```

and **executed** as follows:

```
> primes
[2,3,5,7,11,13,17,19,23,29,31,37,41,43,47,53,59,61,...
```

In [ ]:
primes :: [Int]
primes = seive [2..]

seive :: [Int] -> [Int]
seive (p:xs) = p : seive [x | x <- xs, x `mod` p /= 0]

-- primes

By freeing the generation of primes from the constraint of finiteness, we obtain a **modular** definition on which different **boundary conditions** can be imposed in different situations:

Selecting the first 10 primes:

```
> take 10 primes
[2,3,5,7,11,13,17,19,23,29]
```

Selecting the primes less than 15:

```
> takeWhile (<15) primes
[2,3,5,7,11,13]
```

Lazy evaluation is powerful programming tool!

In [ ]:
take 10 primes

takeWhile (<15) primes

## Fun ICEs

Define a program

```
fibs :: [Integer]
```

that generates the **infinite Fibonacci sequence** `[0, 1, 1, 2, 3, 5, 8, 13, 21, 34, ...` using the following simple procedure:

1. The first two numbers are `0` and `1`;
2. The next is the sum of the previous two;
3. Return to step 2.

In [ ]:
fibs :: [Integer]
fibs = 0 : 1 : zipWith (+) fibs (tail fibs)

take 10 fibs

## Fun ICEs

Define a program

```
fib :: Int -> Integer
```

that calculates the **`n`th** Fibonacci number.

In [ ]:
fib :: Int -> Integer
fib n = fibs !! (n - 1)

fib 5

## Summary

* Declaring types and classes
* Lazy evaluation